In [16]:
using DiffEqBase, OrdinaryDiffEq, DiffEqCallbacks

### ODE
function ode_(du, u, p, t)
    (C,S2,k1,k2) = p
    (S1,S3) = u 

    reaction1 = C * k1 * S1 * S2
    reaction2 = C * k2 * S3

    
    du[1] = -reaction1+reaction2  # dS1/dt
    du[2] =  reaction1-reaction2  # dS3/dt
end

# initial values and parameters
u0 = [0.0, 0.0] # S1, S3
p0 = [1.0, 2.0, 0.75, 0.25] # C, S2, k1, k2
;

In [5]:
## variant 1 : condition tprev == 0
zero_cb_condition1(u,t,integrator) = integrator.tprev == 0.0

zero_cb_assignment1(integrator) = integrator.u .= [1.0,1.0]

zero_cb1 = DiscreteCallback(zero_cb_condition1,zero_cb_assignment1,save_positions=(true,true))

DiscreteCallback{typeof(zero_cb_condition1),typeof(zero_cb_assignment1),typeof(DiffEqBase.INITIALIZE_DEFAULT)}(zero_cb_condition1, zero_cb_assignment1, DiffEqBase.INITIALIZE_DEFAULT, Bool[1, 1])

In [6]:
prob1 = ODEProblem(ode_, u0, (0.,3.), p0, callback=zero_cb1)
sol1 = solve(prob1, Vern9(),
        save_end = false,
        save_everystep = false,
)

retcode: Success
Interpolation: 1st order linear
t: 3-element Array{Float64,1}:
 0.0
 9.999999999999999e-5
 9.999999999999999e-5
u: 3-element Array{Array{Float64,1},1}:
 [0.0, 0.0]
 [0.0, 0.0]
 [1.0, 1.0]

In [17]:
## variant 2 : callabck initialization
zero_cb_condition2(u,t,integrator) = false

zero_cb_assignment2(integrator) = integrator.u .= [1.0,1.0]

zero_cb2 = DiscreteCallback(zero_cb_condition2,zero_cb_assignment2, 
    initialize =  (c,t,u,integrator)->c.affect!(integrator), save_positions=(true,true))

DiscreteCallback{typeof(zero_cb_condition2),typeof(zero_cb_assignment2),var"#11#12"}(zero_cb_condition2, zero_cb_assignment2, var"#11#12"(), Bool[1, 1])

In [18]:
prob2 = ODEProblem(ode_, u0, (0.,3.), p0, callback=zero_cb2)
sol2 = solve(prob1, Vern9(),
        save_end = false,
        save_everystep = false,
)

retcode: Success
Interpolation: 1st order linear
t: 3-element Array{Float64,1}:
 0.0
 9.999999999999999e-5
 9.999999999999999e-5
u: 3-element Array{Array{Float64,1},1}:
 [0.0, 0.0]
 [0.0, 0.0]
 [1.0, 1.0]